In [49]:
using RigidBodyDynamics
using StaticArrays
using MeshCat, MeshCatMechanisms, Blink
using Random
Random.seed!(42);

In [50]:
srcdir = dirname(pathof(RigidBodyDynamics))
urdf = "../../../kortex_description/arms/gen3/7dof/urdf/GEN3_URDF_V12 copy.urdf"
mechanism = parse_urdf(urdf)
state = MechanismState(mechanism)
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: Shoulder_Link, Edge: Actuator1
    Vertex: HalfArm1_Link, Edge: Actuator2
      Vertex: HalfArm2_Link, Edge: Actuator3
        Vertex: ForeArm_Link, Edge: Actuator4
          Vertex: SphericalWrist1_Link, Edge: Actuator5
            Vertex: SphericalWrist2_Link, Edge: Actuator6
              Vertex: Bracelet_Link, Edge: Actuator7
No non-tree joints.

In [51]:
body = findbody(mechanism, "Bracelet_Link")
point = Point3D(default_frame(body), 0., 0, -0.1)

Point3D in "after_Actuator7": [0.0, 0.0, -0.1]

In [52]:
# Create the visualizer
vis = MechanismVisualizer(mechanism, URDFVisuals(urdf))

# Render our target point attached to the robot as a sphere with radius 0.07
setelement!(vis, point, 0.07)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8723
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat/world/Shoulder_Link/HalfArm1_Link/HalfArm2_Link/ForeArm_Link/SphericalWrist1_Link/SphericalWrist2_Link/Bracelet_Link/after_Actuator7/<element> at http://127.0.0.1:8723

In [53]:
open(vis, Window());
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8724
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://127.0.0.1:8724, 11)

In [54]:
function jacobian_transpose_ik!(state::MechanismState,
                               body::RigidBody,
                               point::Point3D,
                               desired::Point3D;
                               α=0.1,
                               iterations=100)
    mechanism = state.mechanism
    world = root_frame(mechanism)

    # Compute the joint path from world to our target body
    p = path(mechanism, root_body(mechanism), body)
    # Allocate the point jacobian (we'll update this in-place later)
    Jp = point_jacobian(state, p, transform(state, point, world))

    q = copy(configuration(state))

    for i in 1:iterations
        # Update the position of the point
        point_in_world = transform(state, point, world)
        # Update the point's jacobian
        point_jacobian!(Jp, state, p, point_in_world)
        # Compute an update in joint coordinates using the jacobian transpose
        Δq = α * Array(Jp)' * (transform(state, desired, world) - point_in_world).v
        # Apply the update
        q .= configuration(state) .+ Δq
        set_configuration!(state, q)
    end
    state
end

jacobian_transpose_ik! (generic function with 1 method)

In [55]:
rand!(state)
set_configuration!(vis, configuration(state))

In [60]:
desired_tip_location = Point3D(root_frame(mechanism), 0.3, 0.3, 0.5)

Point3D in "world": [0.3, 0.3, 0.5]

In [61]:
jacobian_transpose_ik!(state, body, point, desired_tip_location)
set_configuration!(vis, configuration(state))

In [62]:
transform(state, point, root_frame(mechanism))

Point3D in "world": [0.3817078480901187, 0.37894568460683836, 0.5482892291239736]

In [80]:
qs = typeof(configuration(state))[]

SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}[]

In [81]:
for x in range(-0.3, stop=0.1, length=100)
    desired = Point3D(root_frame(mechanism), x, 0.4, 0.5)
    jacobian_transpose_ik!(state, body, point, desired)
    push!(qs, copy(configuration(state)))
end
ts = collect(range(0, stop=10, length=length(qs)))


100-element Vector{Float64}:
  0.0
  0.10101010101010101
  0.20202020202020202
  0.30303030303030304
  0.40404040404040403
  0.5050505050505051
  0.6060606060606061
  0.7070707070707071
  0.8080808080808081
  0.9090909090909091
  1.0101010101010102
  1.1111111111111112
  1.2121212121212122
  ⋮
  8.88888888888889
  8.98989898989899
  9.090909090909092
  9.191919191919192
  9.292929292929292
  9.393939393939394
  9.494949494949495
  9.595959595959595
  9.696969696969697
  9.797979797979798
  9.8989898989899
 10.0

In [82]:
setanimation!(vis, Animation(vis, ts, qs))